In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_score, recall_score

In [2]:
## 设置字符集，防止中文乱码
mpl.rcParams['font.sans-serif']=[u'simHei']
mpl.rcParams['axes.unicode_minus']=False

In [3]:
# jupyter展示图片，非内嵌显示
# tk: 显示出来，inline：内嵌显示，默认为inline
%matplotlib tk

In [4]:
# 1. 文件数据读取
df = pd.read_csv('../data/result_process02', sep=',')
# 如果有某值为nan，进行删除操作
df.dropna(axis=0, how='any', inplace=True)
df.head(10)

,label,from_163,from_tsinghua,from_126,from_yahoo,from_21cn,from_tom,from_12,from_cernet,from_sohu,from_unknown,has_not_date,jieba_cut_content,content_length_type_0,content_length_type_1,content_length_type_2,content_length_type_3,content_sema
0,1.0,0,0,0,0,0,0,0,0,0,1,0,非 财务 纠淼 牟 莆 窆 芾 - （ 沙盘 模拟 ） - - ...,0,0,1,0,8.456151
1,0.0,0,0,0,0,0,0,0,0,0,0,0,讲 的 是 孔子 后人 的 故事 。 一个 老 领导 回到 家乡 ...,0,1,0,0,7.486084
2,1.0,0,0,0,0,0,0,0,0,0,0,0,尊敬 的 贵 公司 ( 财务 / 经理 ) 负责人 您好 ！ 我 ...,0,1,0,0,7.175171
3,1.0,0,0,0,0,0,0,1,0,0,0,0,贵 公司 负责人 ( 经理 / 财务 ） 您好 ： 深圳市 华龙 公...,0,1,0,0,7.565682
4,1.0,0,0,0,0,0,0,0,0,0,0,0,这是 一封 HTML 格式 信件 ！ - - - - - - - ...,0,0,0,1,2.063409
5,1.0,1,0,0,0,0,0,0,0,0,0,0,TO ： 贵 公司 经理 、 财务 您好 ！ 深圳市 春洋 贸易 有...,0,1,0,0,7.143747
6,0.0,0,0,0,0,0,0,0,0,0,0,0,那 他 为什么 不 愿意 起诉 ， 既然 这样 了 ！ 起诉 后 ...,0,1,0,0,4.807568
7,1.0,0,0,0,0,0,1,0,0,0,0,0,尊敬 的 负责人 （ 经理 ／ 财务 ） ： 您好 ！ 我 是 深...,0,1,0,0,6.593684
8,1.0,0,0,0,0,0,0,0,0,0,0,0,您好 以下 是 特别 为 阁下 发 的 香港 信息 ( 图片 ...,0,1,0,0,7.611074
9,0.0,0,0,0,0,0,0,0,0,0,0,0,我 觉得 ， 负债 不要紧 ， 最 重要 的 是 能 负得起 这个 ...,0,1,0,0,7.041340


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64284 entries, 0 to 64618
Data columns (total 18 columns):
label                    64284 non-null float64
from_163                 64284 non-null int64
from_tsinghua            64284 non-null int64
from_126                 64284 non-null int64
from_yahoo               64284 non-null int64
from_21cn                64284 non-null int64
from_tom                 64284 non-null int64
from_12                  64284 non-null int64
from_cernet              64284 non-null int64
from_sohu                64284 non-null int64
from_unknown             64284 non-null int64
has_not_date             64284 non-null int64
jieba_cut_content        64284 non-null object
content_length_type_0    64284 non-null int64
content_length_type_1    64284 non-null int64
content_length_type_2    64284 non-null int64
content_length_type_3    64284 non-null int64
content_sema             64284 non-null float64
dtypes: float64(2), int64(15), object(1)
memory usage: 9.3

In [6]:
# 2. 数据分割
x_train, x_test, y_train, y_test = train_test_split(df[['has_not_date','jieba_cut_content','content_sema']], df['label'], test_size=0.2, random_state=0)
print("训练数据集大小:%d" % x_train.shape[0])
print("测试数据集大小:%d" % x_test.shape[0])

训练数据集大小:51427
测试数据集大小:12857


In [7]:
x_train.head(5)

,has_not_date,jieba_cut_content,content_sema
18809,0,尊敬 的 公司 （ 工厂 ） 经理 负责人 你好 ： 我 公司 是 ...,6.849239
61631,0,声音 变换器 ： 适用 于 不同 型号 的 手机 ， 电话 ， 网络...,7.242064
5822,0,http : / / tengyingge . blogchina . co...,8.289843
5881,0,刚 开始 ， 是 会 有点 不 习惯 的 。 慢慢来 。 ps ：...,7.063309
26337,0,尊敬 的 公司 （ 工厂 ） 经理 负责人 你好 ： 我 公司 是 ...,6.700159


In [8]:
# 3. 开始模型训练
# 3.1 特征工程，将文本数据转换为数值型数据
vectorizer = CountVectorizer()
transfromer = TfidfTransformer(norm='l2', use_idf=True)
svd = TruncatedSVD(n_components=20)
jieba_cut_content = list(x_train['jieba_cut_content'].astype('str'))
vectorizer_model = vectorizer.fit(jieba_cut_content)
df0 = vectorizer_model.transform(jieba_cut_content)
transfromer_model = transfromer.fit(df0)
df1 = transfromer_model.transform(df0)
svd_model = svd.fit(df1)
df2 = svd_model.transform(df1)
data = pd.DataFrame(df2)
print(data.head(5))
print(data.info())

         0         1         2         3         4         5         6   \
0  0.340289  0.220011 -0.237480 -0.039907 -0.070124  0.002535 -0.160571   
1  0.014968  0.064739  0.027004 -0.004650  0.008147  0.007428  0.016296   
2  0.011862  0.072034  0.047003  0.001429 -0.002090  0.035894  0.009201   
3  0.019714  0.117438  0.132448  0.060410 -0.103483 -0.074865 -0.012164   
4  0.346257  0.215335 -0.234032 -0.043097 -0.071025 -0.000022 -0.162419   

         7         8         9         10        11        12        13  \
0 -0.010227  0.201902 -0.010610 -0.199942  0.038707 -0.064122  0.433412   
1  0.046138  0.029614 -0.000293  0.008104  0.023540  0.042377 -0.014192   
2 -0.012208 -0.016863  0.000159  0.001312 -0.003906 -0.002468 -0.003188   
3 -0.005632 -0.027447  0.000417 -0.016917 -0.013160 -0.003024  0.017654   
4 -0.009516  0.199126 -0.010303 -0.195371  0.036965 -0.060836  0.421432   

         14        15        16        17        18        19  
0 -0.477330  0.203009  0.096201 -0

In [9]:
# 3.2 数据合并
data['has_not_date'] = list(x_train['has_not_date'])
data['content_sema'] = list(x_train['content_sema'])
print(data.head(5))
print(data.info())

          0         1         2         3         4         5         6  \
0  0.340289  0.220011 -0.237480 -0.039907 -0.070124  0.002535 -0.160571   
1  0.014968  0.064739  0.027004 -0.004650  0.008147  0.007428  0.016296   
2  0.011862  0.072034  0.047003  0.001429 -0.002090  0.035894  0.009201   
3  0.019714  0.117438  0.132448  0.060410 -0.103483 -0.074865 -0.012164   
4  0.346257  0.215335 -0.234032 -0.043097 -0.071025 -0.000022 -0.162419   

          7         8         9      ...             12        13        14  \
0 -0.010227  0.201902 -0.010610      ...      -0.064122  0.433412 -0.477330   
1  0.046138  0.029614 -0.000293      ...       0.042377 -0.014192  0.012552   
2 -0.012208 -0.016863  0.000159      ...      -0.002468 -0.003188 -0.003193   
3 -0.005632 -0.027447  0.000417      ...      -0.003024  0.017654  0.002108   
4 -0.009516  0.199126 -0.010303      ...      -0.060836  0.421432 -0.467626   

         15        16        17        18        19  has_not_date  \
0  0.

In [10]:
%%time
knn = KNeighborsClassifier(n_neighbors=5)
model = knn.fit(data, y_train)

Wall time: 676 ms


In [11]:
# 4.1 对测试数据进行特征转换
jieba_cut_content_test = list(x_test['jieba_cut_content'].astype('str'))
data_test = pd.DataFrame(svd_model.transform(transfromer_model.transform(vectorizer_model.transform(jieba_cut_content_test))))
data_test['has_not_date'] = list(x_test['has_not_date'])
data_test['content_sema'] = list(x_test['content_sema'])
print(data_test.head(5))
print(data_test.info())

          0         1         2         3         4         5         6  \
0  0.537283  0.009054 -0.087603  0.028877  0.000965 -0.032606 -0.157601   
1  0.210902  0.131678 -0.090069 -0.086976 -0.033301  0.039670  0.006866   
2  0.045151  0.277285  0.233899  0.051107 -0.074067 -0.016335  0.010473   
3  0.032907  0.163069  0.113489  0.002467  0.053689  0.008292  0.004177   
4  0.035278  0.065204  0.026082 -0.023772  0.072907 -0.005845 -0.003324   

          7         8         9      ...             12        13        14  \
0  0.008978 -0.134912  0.010324      ...       0.077265  0.007442 -0.015857   
1 -0.008062  0.011146 -0.000286      ...      -0.026748  0.081783 -0.056804   
2 -0.014665  0.100589 -0.003149      ...      -0.067650 -0.038815 -0.032177   
3 -0.020179  0.118051 -0.002730      ...      -0.061972 -0.058946 -0.013005   
4 -0.007926  0.008235  0.000564      ...      -0.001914 -0.005931 -0.003653   

         15        16        17        18        19  has_not_date  \
0 -0.

In [12]:
# 4.2 对测试数据进行预测
y_predict = model.predict(data_test)

In [13]:
# 5. 效果评估
print("精确率为:%.5f" % precision_score(y_test, y_predict))
print("召回率为:%.5f" % recall_score(y_test, y_predict))
print("F1值为:%.5f" % f1_score(y_test, y_predict))

精确率为:0.97963
召回率为:0.98807
F1值为:0.98383
